In [ ]:
# data = pd.read_csv("bbca_data.csv")

In [ ]:
# data = data.drop(index=[0, 1])
# data.rename(columns={'Price': 'Date'}, inplace=True)


In [ ]:
# data.to_csv("bbca.csv")


In [ ]:
# ===== Cell 1: setup =====
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

# file paths (ubah kalau beda)
PRICE_PATH = "bbca.csv"  # <— file harga baru kamu
FUND_PATH  = "bbca_fundamentals_quarterly_2021_2023.csv"

pd.set_option("display.max_columns", 200)

In [ ]:
# ===== Cell 2: load & clean price =====
def load_price(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    # drop kolom index kalau ada
    for junk in ["Unnamed: 0", "Unnamed: 1"]:
        if junk in df.columns:
            df = df.drop(columns=[junk])

    assert "Date" in df.columns, "Kolom 'Date' wajib ada."
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

    # pastikan numeric
    for c in ["Close","High","Low","Open","Volume"]:
        if c in df.columns and df[c].dtype == object:
            df[c] = df[c].astype(str).str.replace(",", "", regex=False)
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    df = df.dropna(subset=["Date"]).sort_values("Date")
    # deduplicate per tanggal (ambil baris terakhir)
    df = df.groupby("Date", as_index=False).last()
    return df

price = load_price(PRICE_PATH)
price.head(5)

In [ ]:
# ===== Cell 3: technical indicators =====
def add_technicals(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()

    # SMA/EMA
    out["SMA_20"] = out["Close"].rolling(window=20, min_periods=20).mean()
    out["EMA_20"] = out["Close"].ewm(span=20, adjust=False).mean()

    # Bollinger (20, 2)
    bb_w, bb_std = 20, 2.0
    roll_mean = out["Close"].rolling(bb_w, min_periods=bb_w).mean()
    roll_std  = out["Close"].rolling(bb_w, min_periods=bb_w).std()
    out["Bollinger_upper"] = roll_mean + bb_std * roll_std
    out["Bollinger_lower"] = roll_mean - bb_std * roll_std
    out["BB_percentB"]  = (out["Close"] - out["Bollinger_lower"]) / (out["Bollinger_upper"] - out["Bollinger_lower"])
    out["BB_bandwidth"] = (out["Bollinger_upper"] - out["Bollinger_lower"]) / (roll_mean + 1e-12)

    # RSI(14)
    def rsi(series, window=14):
        delta = series.diff()
        up = delta.clip(lower=0)
        down = -delta.clip(upper=0)
        roll_up = up.ewm(alpha=1/window, adjust=False).mean()
        roll_down = down.ewm(alpha=1/window, adjust=False).mean()
        rs = roll_up / (roll_down + 1e-12)
        return 100 - (100 / (1 + rs))
    out["RSI_14"] = rsi(out["Close"], window=14)

    # MACD (12, 26, 9)
    fast, slow, signal = 12, 26, 9
    ema_fast = out["Close"].ewm(span=fast, adjust=False).mean()
    ema_slow = out["Close"].ewm(span=slow, adjust=False).mean()
    out["MACD_line"]   = ema_fast - ema_slow
    out["MACD_signal"] = out["MACD_line"].ewm(span=signal, adjust=False).mean()
    out["MACD_hist"]   = out["MACD_line"] - out["MACD_signal"]

    # log-return & volatilitas cepat
    out["ret_log"] = np.log(out["Close"] / out["Close"].shift(1))
    out["roll_std_5"]  = out["ret_log"].rolling(5,  min_periods=5).std()
    out["roll_std_10"] = out["ret_log"].rolling(10, min_periods=10).std()

    # kalender (cyclical)
    out["dayofweek"] = out["Date"].dt.dayofweek
    out["month"]     = out["Date"].dt.month
    out["day_sin"]   = np.sin(2*np.pi*out["dayofweek"]/7)
    out["day_cos"]   = np.cos(2*np.pi*out["dayofweek"]/7)
    out["mon_sin"]   = np.sin(2*np.pi*(out["month"]-1)/12)
    out["mon_cos"]   = np.cos(2*np.pi*(out["month"]-1)/12)

    return out

price = add_technicals(price)
price.tail(3)

In [ ]:
# ===== Cell 3b (ADD): lag & momentum features =====
def add_lags_and_momentum(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()

    # set lag yang umum untuk daily
    lag_list = [1, 2, 3, 5, 10, 20]

    # kolom yang dilag
    base_cols = [
        "ret_log", "Close", "Volume",
        "RSI_14", "MACD_line", "MACD_signal",
        "BB_percentB", "BB_bandwidth",
        "roll_std_5", "roll_std_10",
        "SMA_20", "EMA_20"
    ]
    for c in base_cols:
        if c in out.columns:
            for L in lag_list:
                out[f"{c}_lag{L}"] = out[c].shift(L)

    # momentum / smoothing return
    for w in [5, 10, 20]:
        if "ret_log" in out.columns:
            out[f"ret_log_sma_{w}"] = out["ret_log"].rolling(w, min_periods=w).mean()
            out[f"ret_log_ema_{w}"] = out["ret_log"].ewm(span=w, adjust=False).mean()
            out[f"ret_log_cum_{w}"] = out["ret_log"].rolling(w, min_periods=w).sum()

    return out

price = add_lags_and_momentum(price)

In [ ]:
# ===== Cell 4: fundamentals QoQ/YoY to daily =====
def quarter_end_date(q_label: str) -> str:
    q_label = (q_label or "").upper().strip()
    mapping = {"Q1":"03-31","Q2":"06-30","Q3":"09-30","Q4":"12-31"}
    return mapping.get(q_label, "12-31")

def load_fundamentals(path: str) -> pd.DataFrame:
    f = pd.read_csv(path)
    f.columns = [c.strip() for c in f.columns]
    assert "Periode" in f.columns and "Quartal" in f.columns, "Kolom 'Periode' & 'Quartal' wajib ada."

    years = f["Periode"].astype(str).str.extract(r"(\d{4})")[0]
    qend  = f["Quartal"].astype(str).map(quarter_end_date)
    f["Date"] = pd.to_datetime(years + "-" + qend, errors="coerce")

    # bersihkan persen & koma -> numeric
    for c in f.columns:
        if c not in ["Date","Periode","Quartal"]:
            if f[c].dtype == object:
                f[c] = (f[c].astype(str)
                              .str.replace("%","", regex=False)
                              .str.replace(",","", regex=False))
            f[c] = pd.to_numeric(f[c], errors="coerce")

    f = f.dropna(subset=["Date"]).sort_values("Date")

    num_cols = [c for c in f.columns if c!="Date" and f[c].dtype.kind in "fcbiu"]
    fq = f[["Date"] + num_cols].copy()

    # ΔQoQ & ΔYoY dihitung di level KUARTAL
    for col in num_cols:
        fq[f"{col}_QoQ"] = (fq[col] - fq[col].shift(1)) / fq[col].shift(1).abs()
        fq[f"{col}_YoY"] = (fq[col] - fq[col].shift(4)) / fq[col].shift(4).abs()

    return fq

def fundamentals_to_daily(fq: pd.DataFrame, price_dates: pd.Series) -> pd.DataFrame:
    daily = fq.set_index("Date").sort_index().ffill().bfill()
    daily = daily.reindex(price_dates).ffill().bfill().reset_index().rename(columns={"index":"Date"})
    return daily

fund_q = load_fundamentals(FUND_PATH)
fund_daily = fundamentals_to_daily(fund_q, price["Date"])

# merge semua
data = price.merge(fund_daily, on="Date", how="left")
print("Rows before target:", len(data))
data.head(3)

In [ ]:
# --- CEK JUMLAH DATA RAW & MERGE (SESUAI NAMA VARIABEL NOTEBOOK) ---

def _rng(df, date_col="Date"):
    try:
        dmin, dmax = df[date_col].min(), df[date_col].max()
        return f"{dmin} → {dmax}"
    except Exception:
        return "-"

print("=== RAW TEKNIKAL (price / OHLCV) ===")
print("rows:", len(price), "| cols:", len(price.columns), "| range:", _rng(price))
print("kolom:", sorted(price.columns.tolist())[:12], "...")

print("\n=== RAW FUNDAMENTAL KUARTALAN (fund_q) ===")
print("rows:", len(fund_q), "| cols:", len(fund_q.columns))
print("kolom:", sorted(fund_q.columns.tolist())[:12], "...")

print("\n=== FUNDAMENTAL HARIAN (fund_daily) — hasil konversi/expand ===")
print("rows:", len(fund_daily), "| cols:", len(fund_daily.columns), "| range:", _rng(fund_daily))
print("kolom:", sorted(fund_daily.columns.tolist())[:12], "...")

print("\n=== HASIL MERGE (data = price ⋈ fund_daily) — sebelum target ===")
print("Rows before target:", len(data), "| cols:", len(data.columns), "| range:", _rng(data))


In [ ]:
# ===== Cell 5: target =====
# Target = Close(t+1) — gampang diganti kalau mau pakai return
data["target"] = data["Close"].shift(-1)

# (alternatif) target return besok:
# data["target"] = data["ret_log"].shift(-1)

data = data.dropna().reset_index(drop=True)
print("Rows after dropna:", len(data))
data.head(3)

In [ ]:
# ===== Cell 6: split & scaling =====
def time_split(df: pd.DataFrame, train_ratio=0.70, val_ratio=0.15):
    n = len(df)
    train_end = int(n*train_ratio)
    val_end   = int(n*(train_ratio+val_ratio))
    train = df.iloc[:train_end].copy()
    val   = df.iloc[train_end:val_end].copy()
    test  = df.iloc[val_end:].copy()
    return train, val, test

train, val, test = time_split(data, train_ratio=0.70, val_ratio=0.15)

exclude = {"Date","target"}
feature_cols = [c for c in data.columns if c not in exclude and data[c].dtype.kind in "fcbiu"]
len(feature_cols), feature_cols[:12]

In [ ]:
# ===== Cell 7: baseline & models =====
scaler = StandardScaler()
X_train = scaler.fit_transform(train[feature_cols].values)
X_val   = scaler.transform(val[feature_cols].values)
X_test  = scaler.transform(test[feature_cols].values)

y_train = train["target"].values
y_val   = val["target"].values
y_test  = test["target"].values

def evaluate_regression(y_true, y_pred, name=""):
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mae  = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred) if len(np.unique(y_true))>1 else np.nan
    print(f"[{name}] RMSE={rmse:.6f}  MAE={mae:.6f}  R²={r2:.6f}")
    return rmse, mae, r2

def price_naive_baseline(y_true):
    # baseline harga: t+1 = t
    y_true_b = y_true[1:]
    y_naive  = np.roll(y_true, 1)[1:]
    return mean_squared_error(y_true_b, y_naive, squared=False)

print("=== BASELINE (Price Naive) ===")
print("VAL baseline RMSE :", price_naive_baseline(y_val))
print("TEST baseline RMSE:", price_naive_baseline(y_test))

ridge = Ridge(alpha=1.0, random_state=0)
ridge.fit(X_train, y_train)
y_val_ridge  = ridge.predict(X_val)
y_test_ridge = ridge.predict(X_test)
print("\n=== RIDGE ===")
rmse_val_ridge, _, _ = evaluate_regression(y_val, y_val_ridge, "Ridge (val)")
rmse_test_ridge, _, _ = evaluate_regression(y_test, y_test_ridge, "Ridge (test)")

rf = RandomForestRegressor(n_estimators=400, max_depth=None, min_samples_leaf=3, random_state=0, n_jobs=-1)
rf.fit(X_train, y_train)
y_val_rf  = rf.predict(X_val)
y_test_rf = rf.predict(X_test)
print("\n=== RANDOM FOREST ===")
rmse_val_rf, _, _ = evaluate_regression(y_val, y_val_rf, "RF (val)")
rmse_test_rf, _, _ = evaluate_regression(y_test, y_test_rf, "RF (test)")

best_name = "ridge" if rmse_val_ridge < rmse_val_rf else "rf"
y_test_best = y_test_ridge if best_name=="ridge" else y_test_rf
rmse_best = mean_squared_error(y_test, y_test_best, squared=False)
nrmse_std = rmse_best / (np.std(y_test) + 1e-12)

print(f"\nBest on VAL: {best_name}")
print(f"TEST RMSE: {rmse_best:.6f}  |  TEST NRMSE(std): {nrmse_std:.3f}")
print(f"TEST baseline RMSE: {price_naive_baseline(y_test):.6f}")

In [ ]:
# ===== Cell 8: plot diag =====
plt.figure(figsize=(5,5))
plt.scatter(y_test, y_test_best, s=8, alpha=0.6)
mn, mx = np.nanmin([y_test.min(), y_test_best.min()]), np.nanmax([y_test.max(), y_test_best.max()])
plt.plot([mn, mx], [mn, mx], linewidth=2)
plt.xlabel("Actual Close (t+1)")
plt.ylabel("Predicted")
plt.title(f"Pred vs Actual (Test) - {best_name.upper()}")
plt.grid(True)
plt.show()

In [ ]:
# ===== Cell (baru): lag features sebelum df_tft dipakai =====
lag_cols = [
    "Close","Volume","SMA_20","EMA_20","BB_percentB","BB_bandwidth",
    "RSI_14","MACD_line","MACD_signal","MACD_hist","ret_log",
    "roll_std_5","roll_std_10"
]
for c in lag_cols:
    if c in price.columns:
        price[c+"_lag1"] = price[c].shift(1)
        price[c+"_lag5"] = price[c].shift(5)

# setelah ini lanjut merge ke fund_daily seperti biasa

In [ ]:
# ===== [PATCH] Cell 10: siapkan data untuk TFT (dengan cleanup NaN) =====
from copy import deepcopy

fund_daily_shifted = fund_daily.copy()
shift_cols = [c for c in fund_daily_shifted.columns if c != "Date"]
fund_daily_shifted[shift_cols] = fund_daily_shifted[shift_cols].shift(7)

df_tft = deepcopy(price.merge(fund_daily_shifted, on="Date", how="left"))

# fitur kalender (kalau belum ada di df_tft karena kamu hitungnya di 'price')
if "day_sin" not in df_tft.columns:
    df_tft["dayofweek"] = df_tft["Date"].dt.dayofweek
    df_tft["month"]     = df_tft["Date"].dt.month
    df_tft["day_sin"]   = np.sin(2*np.pi*df_tft["dayofweek"]/7)
    df_tft["day_cos"]   = np.cos(2*np.pi*df_tft["dayofweek"]/7)
    df_tft["mon_sin"]   = np.sin(2*np.pi*(df_tft["month"]-1)/12)
    df_tft["mon_cos"]   = np.cos(2*np.pi*(df_tft["month"]-1)/12)

# daftar known/unknown reals — SAMA seperti yang kamu pakai
base_known = ["day_sin","day_cos","mon_sin","mon_cos"]

fund_level_candidates = ["NPL_Gross (%)","CAR/KPMM (%)","CET-1(%)",
                         "NII_Midrupiah","Fee_based_Income","CKPN_Midrupiah"]
fund_level = [c for c in fund_level_candidates if c in df_tft.columns]
fund_delta = [c for c in df_tft.columns if c.endswith("_QoQ") or c.endswith("_YoY")]

known_reals   = [c for c in (base_known + fund_level + fund_delta) if c in df_tft.columns]
unknown_reals = [c for c in [
    "Close","Volume","SMA_20","EMA_20",
    "Bollinger_upper","Bollinger_lower","BB_percentB","BB_bandwidth",
    "RSI_14","MACD_line","MACD_signal","MACD_hist",
    "ret_log","roll_std_5","roll_std_10"
] if c in df_tft.columns]

# === CLEANUP: drop baris yang masih punya NaN di fitur real + target Close
cols_required = list(dict.fromkeys(known_reals + unknown_reals + ["Close"]))  # unique, preserve order
df_tft = df_tft.dropna(subset=[c for c in cols_required if c in df_tft.columns]).reset_index(drop=True)

# tambahkan kolom wajib TFT
df_tft["group_id"] = "BBCA"
df_tft["time_idx"] = np.arange(len(df_tft))

# split waktu untuk TFT (karena kita drop warm-up rows, index lama nggak 1:1)
def time_split_df(df, train_ratio=0.70, val_ratio=0.15):
    n = len(df)
    train_end = int(n*train_ratio)
    val_end   = int(n*(train_ratio+val_ratio))
    return df.iloc[:train_end].copy(), df.iloc[train_end:val_end].copy(), df.iloc[val_end:].copy()

train_df, val_df, test_df = time_split_df(df_tft)
print(train_df.shape, val_df.shape, test_df.shape)

In [ ]:
# ===== Cell 11 (REPLACE): TimeSeriesDataSet, target=ret_log, H=1 =====
from pytorch_forecasting.data import TimeSeriesDataSet
from pytorch_forecasting.data.encoders import GroupNormalizer
import numpy as np

# --- horizon & window ---
MAX_ENCODER_LENGTH = 90      # memori encoder (naikkan kalau kuat)
MAX_PRED_LENGTH    = 1       # single-step dulu

# --- known/unknown reals dasar ---
base_known = ["time_idx","day_sin","day_cos","mon_sin","mon_cos"]

fund_level_candidates = ["NPL_Gross (%)","CAR/KPMM (%)","CET-1(%)",
                         "NII_Midrupiah","Fee_based_Income","CKPN_Midrupiah"]
fund_level = [c for c in fund_level_candidates if c in df_tft.columns]
fund_delta = [c for c in df_tft.columns if c.endswith("_QoQ") or c.endswith("_YoY")]

known_reals = [c for c in (base_known + fund_level + fund_delta) if c in df_tft.columns]
unknown_reals = [c for c in [
    "Close","Volume","SMA_20","EMA_20",
    "Bollinger_upper","Bollinger_lower","BB_percentB","BB_bandwidth",
    "RSI_14","MACD_line","MACD_signal","MACD_hist",
    "ret_log","roll_std_5","roll_std_10"
] if c in df_tft.columns]

# --- tambahkan semua kolom lag/momentum yang tersedia ---
lag_like = [c for c in df_tft.columns
            if ("_lag" in c) or ("ret_log_sma_" in c) or ("ret_log_ema_" in c) or ("ret_log_cum_" in c)]
unknown_reals = sorted(list(set(unknown_reals + lag_like)))

# --- SHRINK: batasi feature set biar gak OOM ---
must_keep_base = [
    "Close","Volume","SMA_20","EMA_20",
    "BB_percentB","BB_bandwidth",
    "RSI_14","MACD_line","MACD_signal","MACD_hist",
    "ret_log","roll_std_5","roll_std_10",
]
keep_lags = [
    "ret_log_lag1","ret_log_lag5",
    "Close_lag1","Volume_lag1",
    "RSI_14_lag1","MACD_line_lag1","BB_percentB_lag1",
]
keep_momentum = ["ret_log_sma_5","ret_log_ema_5","ret_log_cum_5",
                 "ret_log_ema_10"]   # <-- tambah 1 ini saja dulu (aman memori)
allowed_unknown = set(must_keep_base + keep_lags + keep_momentum)
unknown_reals = [c for c in unknown_reals if c in allowed_unknown]

# --- CLEAN: inf->NaN, drop rows yang masih NaN di kolom wajib ---
df_tft = df_tft.replace([np.inf, -np.inf], np.nan)

required_cols = sorted(set(known_reals + unknown_reals + ["ret_log"]))  # target=ret_log
required_cols = [c for c in required_cols if c in df_tft.columns]

df_tft = df_tft.dropna(subset=required_cols).reset_index(drop=True)

# --- time_idx harus konsisten setelah drop ---
df_tft["time_idx"] = np.arange(len(df_tft))

# --- re-split setelah bersih ---
def time_split_df(df, train_ratio=0.70, val_ratio=0.15):
    n = len(df); tr = int(n*train_ratio); va = int(n*(train_ratio+val_ratio))
    return df.iloc[:tr].copy(), df.iloc[tr:va].copy(), df.iloc[va:].copy()

train_df, val_df, test_df = time_split_df(df_tft)
print("Shapes after clean/shrink:", train_df.shape, val_df.shape, test_df.shape)
print("known_reals:", len(known_reals), "| unknown_reals:", len(unknown_reals))

# --- bangun dataset TFT ---
training = TimeSeriesDataSet(
    train_df,
    time_idx="time_idx",
    target="ret_log",                         # target = log-return
    group_ids=["group_id"],
    max_encoder_length=MAX_ENCODER_LENGTH,
    max_prediction_length=MAX_PRED_LENGTH,
    static_categoricals=["group_id"],
    time_varying_known_reals=sorted(set(known_reals + ["time_idx"])),
    time_varying_unknown_reals=unknown_reals,
    target_normalizer=GroupNormalizer(groups=["group_id"]),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

validation = TimeSeriesDataSet.from_dataset(training, val_df, predict=True, stop_randomization=True)
testing    = TimeSeriesDataSet.from_dataset(training, test_df, predict=True, stop_randomization=True)

# VAL untuk training/early-stopping: predict=False (sliding windows)
val_ds_train = TimeSeriesDataSet.from_dataset(training, val_df, predict=False, stop_randomization=True)
val_dataloader_train = val_ds_train.to_dataloader(train=False, batch_size=16, num_workers=0)

# --- dataloaders (batch kecil biar aman memori) ---
train_dataloader = training.to_dataloader(train=True,  batch_size=16, num_workers=0)
val_dataloader   = validation.to_dataloader(train=False, batch_size=16, num_workers=0)
test_dataloader  = testing.to_dataloader(train=False, batch_size=16, num_workers=0)

In [ ]:
# ===== Cell 12 (FINAL REPLACE): Train TFT + Robust Manual Load (no load_from_checkpoint) =====
import torch, inspect, importlib, pandas as pd
import pytorch_forecasting as pf
from pytorch_forecasting import TemporalFusionTransformer
from pytorch_forecasting.metrics import MAE
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

device = "cuda" if torch.cuda.is_available() else "cpu"
seed_everything(42, workers=True)

# --- 1) Definisi model dari dataset (sama persis utk reload) ---
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=1e-3,          # lebih kalem
    hidden_size=64,
    attention_head_size=2,
    dropout=0.1,
    loss=MAE(),                  # pakai MAE
    log_interval=-1,
    reduce_on_plateau_patience=4,
)

# --- 2) Callbacks & Trainer ---
early_stop = EarlyStopping(monitor="val_loss", patience=8, mode="min")
ckpt = ModelCheckpoint(
    dirpath="checkpoints",
    filename="tft-{epoch:02d}-{val_loss:.4f}",
    monitor="val_loss", save_top_k=1, mode="min"
)

trainer = Trainer(
    max_epochs=80,
    accelerator="auto",
    devices=1,
    gradient_clip_val=0.1,
    callbacks=[early_stop, ckpt],
    log_every_n_steps=1,
    enable_progress_bar=True,
    logger=False,
)

# --- 3) Train ---
trainer.fit(tft, train_dataloader, val_dataloader_train)

best_path = ckpt.best_model_path
print("Best checkpoint:", best_path)

# ===== 4) Robust manual load (hindari UnpicklingError PyTorch 2.6) =====
def _resolve(fqn: str):
    """Import object by fully-qualified name; return None jika tidak ada."""
    try:
        mod, name = fqn.rsplit(".", 1)
        return getattr(importlib.import_module(mod), name)
    except Exception:
        return None

# Allowlist encoder pytorch-forecasting + tipe/func pandas yang sering tersimpan di ckpt
allowed = []
enc_mod = pf.data.encoders
for name in ["GroupNormalizer","EncoderNormalizer","NaNLabelEncoder","MultiNormalizer","TorchNormalizer"]:
    if hasattr(enc_mod, name) and inspect.isclass(getattr(enc_mod, name)):
        allowed.append(getattr(enc_mod, name))

for fqn in [
    "pandas.core.frame.DataFrame",
    "pandas.core.series.Series",
    "pandas.core.indexes.base.Index",
    "pandas.core.indexes.range.RangeIndex",
    "pandas.core.indexes.datetimes.DatetimeIndex",
    "pandas.core.indexes.multi.MultiIndex",
    "pandas.core.internals.managers.BlockManager",
    "pandas.core.internals.blocks.Block",
    "pandas.core.arrays.categorical.Categorical",
    "pandas._libs.internals._unpickle_block",   # beberapa versi pandas butuh ini
]:
    obj = _resolve(fqn)
    if obj is not None:
        allowed.append(obj)

# --- 5) Muat ckpt sebagai dict & inject state_dict ke model baru (AMAN) ---
if best_path:
    # (a) load raw checkpoint dict (weights_only=False) di dalam safe context
    with torch.serialization.safe_globals(allowed):
        ckpt_raw = torch.load(best_path, map_location=device, weights_only=False)

    # (b) bangun ulang arsitektur yg identik
    tft_loaded = TemporalFusionTransformer.from_dataset(
        training,
        learning_rate=1e-3,
        hidden_size=64,
        attention_head_size=2,
        dropout=0.1,
        loss=MAE(),
        log_interval=-1,
        reduce_on_plateau_patience=4,
    )

    # (c) ambil state_dict (kompatibel berbagai format ckpt)
    sd = ckpt_raw.get("state_dict", ckpt_raw)

    # (d) load bobot
    missing, unexpected = tft_loaded.load_state_dict(sd, strict=False)
    if missing or unexpected:
        print("[INFO] load_state_dict warnings:", {"missing": missing, "unexpected": unexpected})

    tft = tft_loaded.to(device).eval()
    print("✅ Checkpoint loaded via manual state_dict (no load_from_checkpoint)")
else:
    print("[WARN] best_model_path kosong; cek callback ModelCheckpoint/monitor")


In [ ]:
tft.load_state_dict(torch.load("tft_model.pth"))

In [ ]:
# ===== Cell 13 (REPLACE): evaluasi untuk target=log-return =====
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Dataset & dataloader KHUSUS evaluasi (sliding windows), tidak dipakai saat training
eval_val_ds  = TimeSeriesDataSet.from_dataset(training, val_df,  predict=False, stop_randomization=True)
eval_test_ds = TimeSeriesDataSet.from_dataset(training, test_df, predict=False, stop_randomization=True)

eval_val_loader  = eval_val_ds.to_dataloader(train=False, batch_size=64, num_workers=0)
eval_test_loader = eval_test_ds.to_dataloader(train=False, batch_size=64, num_workers=0)

def _to_np(t):
    a = t.detach().cpu().numpy()
    if a.ndim == 3 and a.shape[-1] == 1:
        a = a[..., 0]   # (N,H,1) -> (N,H)
    return a

def eval_tft_returns(dataloader, name="VAL"):
    # pakai mode="raw" supaya kita bisa ambil decoder_target & decoder_lengths
    raw, x = tft.predict(dataloader, mode="raw", return_x=True)

    y_pred_all = _to_np(raw["prediction"])          # (N,H)
    y_true_all = _to_np(x["decoder_target"])        # (N,H)
    dec_len    = _to_np(x["decoder_lengths"]).astype(int).reshape(-1)  # (N,)

    # pastikan 2D
    if y_pred_all.ndim == 1: y_pred_all = y_pred_all[:, None]
    if y_true_all.ndim == 1: y_true_all = y_true_all[:, None]

    N, H = y_true_all.shape

    # mask horizon valid → flatten
    mask   = (np.arange(H)[None, :] < dec_len[:, None])
    y_true = y_true_all[mask].astype(float)
    y_pred = y_pred_all[mask].astype(float)

    # baseline untuk RETURN = 0 (arti: ekspektasi return = 0)
    y_base = np.zeros_like(y_true)

    # filter finite
    m = np.isfinite(y_true) & np.isfinite(y_pred)
    y_true = y_true[m]; y_pred = y_pred[m]; y_base = y_base[m]

    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mae  = mean_absolute_error(y_true, y_pred)

    # R² bisa NaN kalau var(y_true)=0
    r2 = r2_score(y_true, y_pred) if len(np.unique(y_true)) > 1 else np.nan

    # NRMSE by std — kalau std ≈ 0, set NaN biar gak misleading
    std = np.std(y_true)
    nrmse_std = rmse / std if std > 1e-8 else np.nan

    rmse_baseline = mean_squared_error(y_true, y_base, squared=False)

    print(f"\n📊 {name} — TFT (target=log-return)")
    print(f"RMSE={rmse:.6f} | MAE={mae:.6f} | R²={r2:.4f} | baseline RMSE={rmse_baseline:.6f} | NRMSE(std)={(nrmse_std if nrmse_std==nrmse_std else float('nan')):.3f}")

    return {
        "y_true_all": y_true_all,  # <-- dibalikin lagi utk Cell 14
        "y_pred_all": y_pred_all,
        "dec_len": dec_len,
        "rmse": rmse,
        "rmse_baseline": rmse_baseline,
        "mae": mae,
        "r2": r2,
        "nrmse_std": nrmse_std,
    }

val_eval  = eval_tft_returns(eval_val_loader,  "VAL")
test_eval = eval_tft_returns(eval_test_loader, "TEST")

In [ ]:
# ===== Cell 14 (REPLACE): per-horizon metrics langsung dari model =====
def per_horizon_from_model(dataloader, title="VAL"):
    raw, x = tft.predict(dataloader, mode="raw", return_x=True)

    y_pred_all = _to_np(raw["prediction"])
    y_true_all = _to_np(x["decoder_target"])
    dec_len    = _to_np(x["decoder_lengths"]).astype(int).reshape(-1)

    if y_pred_all.ndim == 1: y_pred_all = y_pred_all[:, None]
    if y_true_all.ndim == 1: y_true_all = y_true_all[:, None]

    H = y_true_all.shape[1]
    print(f"\n🔎 Per-horizon — {title} (H={H})")

    for h in range(H):
        valid = dec_len > h
        if valid.sum() == 0: 
            continue
        yt = y_true_all[valid, h].astype(float)
        yp = y_pred_all[valid, h].astype(float)
        m  = np.isfinite(yt) & np.isfinite(yp)
        yt, yp = yt[m], yp[m]
        if len(yt) == 0: 
            continue

        rmse_h = mean_squared_error(yt, yp, squared=False)
        r2_h   = r2_score(yt, yp) if len(np.unique(yt))>1 else np.nan
        print(f"H{h+1}: RMSE={rmse_h:.6f} | R²={r2_h:.4f}")

per_horizon_from_model(eval_val_loader,  "VAL")
per_horizon_from_model(eval_test_loader, "TEST")

In [ ]:
# ===== Cell 15: interpretability — variable importance =====
import numpy as np
import matplotlib.pyplot as plt

raw_val, x_val = tft.predict(val_dataloader, mode="raw", return_x=True)
interp = tft.interpret_output(raw_val, reduction="sum")  # global

def plot_vi(names, scores, title):
    order = np.argsort(scores)[::-1]
    names  = [names[i] for i in order]
    scores = [scores[i] for i in order]
    plt.figure(figsize=(6,4))
    plt.barh(range(len(scores)), scores[::-1])
    plt.yticks(range(len(scores)), names[::-1])
    plt.title(title); plt.xlabel("Importance")
    plt.grid(True, axis="x", alpha=0.3)
    plt.show()

if "encoder_variable_importance" in interp and "encoder_variables" in interp:
    enc_names  = interp["encoder_variables"]
    enc_scores = interp["encoder_variable_importance"].detach().cpu().numpy()
    plot_vi(enc_names, enc_scores, "Encoder variable importance")

if "decoder_variable_importance" in interp and "decoder_variables" in interp:
    dec_names  = interp["decoder_variables"]
    dec_scores = interp["decoder_variable_importance"].detach().cpu().numpy()
    plot_vi(dec_names, dec_scores, "Decoder variable importance")

if "static_variables" in interp and "static_variable_importance" in interp:
    st_names  = interp["static_variables"]
    st_scores = interp["static_variable_importance"].detach().cpu().numpy()
    plot_vi(st_names, st_scores, "Static variable importance")

In [ ]:
# ===== Cell 16 (REPLACE): Interpretabilitas TFT (aman) =====
import numpy as np

# pakai loader evaluasi yang predict=False biar sampel banyak (sudah dibuat di Cell 13)
raw_val, x_val = tft.predict(eval_val_loader, mode="raw", return_x=True)

# beberapa versi bisa balik list; normalkan ke dict
if isinstance(raw_val, list):
    # gabung batch pertama saja untuk interpretasi cepat
    raw_val = raw_val[0]

# 1) Variable importance dari Variable Selection Network
#    reduction harus "mean" atau "sum" (bukan None)
interpv = tft.interpret_output(raw_val, reduction="mean")   # <-- FIX di sini

def _agg_importance(a):
    a = np.array(a)
    # rata-rata absolut di semua axis kecuali axis variabel (terakhir)
    if a.ndim == 1:
        return np.abs(a)
    axes = tuple(range(a.ndim - 1))
    return np.nanmean(np.abs(a), axis=axes)

enc_imp = _agg_importance(interpv.get("encoder_variables"))
dec_imp = _agg_importance(interpv.get("decoder_variables"))

# ambil nama variabel: coba dari hparams, fallback ke dataset
try:
    var_names = list(tft.hparams.x_reals)
except Exception:
    try:
        # beberapa versi PF simpan di dataset
        var_names = list(training.reals)
    except Exception:
        var_names = [f"var_{i}" for i in range(len(enc_imp))]

def _topk(names, scores, k=15):
    idx = np.argsort(scores)[::-1][:min(k, len(scores))]
    return [(names[i], float(scores[i])) for i in idx]

top_enc = _topk(var_names, enc_imp, k=15)
top_dec = _topk(var_names, dec_imp, k=15)

print("\n🔎 Encoder variable importance (top 15):")
for n,s in top_enc:
    print(f"{n:20s}  {s:.6f}")

print("\n🔎 Decoder variable importance (top 15):")
for n,s in top_dec:
    print(f"{n:20s}  {s:.6f}")

# 2) Attention over encoder timesteps (lag mana yang paling diperhatikan)
attn = interpv.get("attention", None)
if attn is not None:
    A = np.array(attn)
    # bentuk umum: (batch, heads, dec_time, enc_time)
    # rata-rata semua kecuali dim terakhir (enc_time)
    axes = tuple(range(A.ndim - 1))
    enc_time_scores = np.nanmean(A, axis=axes) if A.ndim > 1 else A
    # ranking lag encoder (0=posisi paling lama di encoder, -1=terdekat)
    enc_steps = np.arange(enc_time_scores.shape[-1])
    idx = np.argsort(enc_time_scores)[::-1][:10]
    print("\n🎯 Top attention encoder steps (terbesar -> terkecil):")
    for i in idx:
        print(f"encoder_step={int(enc_steps[i])}, score={float(enc_time_scores[i]):.6f}")
else:
    print("\n(i) Attention tidak tersedia pada objek output ini — tetap bisa pakai variable importance di atas.")

In [ ]:
# ===== Cell 17: kumpulkan prediksi & align ke tanggal =====
import numpy as np
import pandas as pd

# helper: kalau _to_np belum ada (mis. kamu clear kernel), define lagi
try:
    _to_np
except NameError:
    def _to_np(t):
        a = t.detach().cpu().numpy()
        if a.ndim == 3 and a.shape[-1] == 1:
            a = a[..., 0]
        return a

def collect_pred_df(dataloader, name="TEST"):
    raw, x, idx = tft.predict(dataloader, mode="raw", return_x=True, return_index=True)

    # (N,H) flatten -> 1D
    y_pred_all = _to_np(raw["prediction"])
    y_true_all = _to_np(x["decoder_target"])
    if y_pred_all.ndim == 1: y_pred_all = y_pred_all[:, None]
    if y_true_all.ndim == 1: y_true_all = y_true_all[:, None]
    N, H = y_true_all.shape

    # ambil time_idx horizon-0 (H=1 → satu kolom)
    # coba dari x["decoder_time_idx"], fallback ke idx["time_idx"]
    if "decoder_time_idx" in x:
        t_idx = _to_np(x["decoder_time_idx"]).astype(int)
        t_idx_h0 = t_idx[:, 0] if t_idx.ndim == 2 else t_idx.reshape(-1)
    else:
        # idx bisa DataFrame dengan kolom 'time_idx' per sampel
        if isinstance(idx, (pd.DataFrame, pd.Series)) and "time_idx" in idx:
            t_idx_h0 = idx["time_idx"].to_numpy().astype(int).reshape(-1)
        else:
            # fallback konservatif: numbering urut (kurang ideal)
            t_idx_h0 = np.arange(N)

                # flatten ke 1D (H=1)
    y_pred = y_pred_all[:, 0]
    y_true = y_true_all[:, 0]

    # map ke Date & Close dari df_tft
    map_df = df_tft[["time_idx","Date","Close"]].copy()
    map_df["Close_prev"] = map_df["Close"].shift(1)
    t2date = dict(zip(map_df["time_idx"], map_df["Date"]))
    t2close = dict(zip(map_df["time_idx"], map_df["Close"]))
    t2close_prev = dict(zip(map_df["time_idx"], map_df["Close_prev"]))

    df = pd.DataFrame({
        "split": name,
        "time_idx": t_idx_h0,
        "ret_true": y_true,
        "ret_pred": y_pred,
    })
    df["Date"] = df["time_idx"].map(t2date)
    df["Close_true"] = df["time_idx"].map(t2close)            # Close(t)
    df["Close_prev"] = df["time_idx"].map(lambda t: t2close_prev.get(t))  # Close(t-1)

    # prediksi harga H1 dari return: P_t_hat = P_{t-1} * exp(ret_pred)
    df["Close_pred"] = df["Close_prev"] * np.exp(df["ret_pred"])
    # baseline harga: hold-last
    df["Close_base"] = df["Close_prev"]
    df["year"] = df["Date"].dt.year
    # buang baris yang masih NaN (awal seri biasanya)
    df = df.dropna(subset=["Date","Close_true","Close_prev","Close_pred"]).reset_index(drop=True)
    return df


df_val  = collect_pred_df(eval_val_loader,  "VAL")
df_test = collect_pred_df(eval_test_loader, "TEST")

print(df_val.head(3))
print(df_test.head(3))

In [ ]:
# # --- SAVE TRAINED MODEL ---
# print("Saving TFT model as tft_model.pth ...")
# import torch
# torch.save(tft.state_dict(), "tft_model.pth")
# print("All files saved successfully!")

In [ ]:
# --- SAVE PREDICTION RESULT ---
print("Saving df_test as predicted_tft.csv ...")
df_test.to_csv("predicted_tft.csv", index=False)


In [ ]:
# ===== Cell 18: plot garis harga aktual vs prediksi (TEST) =====
import matplotlib.pyplot as plt

dfp = df_test.sort_values("Date")
plt.figure(figsize=(12,5))
plt.plot(dfp["Date"], dfp["Close_true"], label="Actual Close")
plt.plot(dfp["Date"], dfp["Close_pred"], label="Predicted Close (H1)")
plt.title("BBCA — Actual vs Predicted Close (H1) — TEST")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# ===== Cell 19: scatter return pred vs aktual (TEST) =====
import matplotlib.pyplot as plt
import numpy as np

dfr = df_test.dropna(subset=["ret_true","ret_pred"])
x = dfr["ret_true"].values
y = dfr["ret_pred"].values

plt.figure(figsize=(6,6))
plt.scatter(x, y, alpha=0.5, s=16)
lims = [min(x.min(), y.min()), max(x.max(), y.max())]
plt.plot(lims, lims, linestyle="--")  # y=x
plt.title("Predicted vs Actual log-return (H1) — TEST")
plt.xlabel("Actual ret_log")
plt.ylabel("Predicted ret_log")
plt.tight_layout()
plt.show()

# korelasi sederhana
corr = np.corrcoef(x, y)[0,1]
print(f"Correlation (TEST) = {corr:.4f}")

In [ ]:
# ===== Cell 20: residual harga (TEST) =====
import matplotlib.pyplot as plt
res = (df_test["Close_true"] - df_test["Close_pred"]).values

plt.figure(figsize=(12,3))
plt.plot(df_test["Date"], res)
plt.axhline(0, linestyle="--")
plt.title("Residuals (Actual - Predicted) Close — TEST")
plt.xlabel("Date")
plt.ylabel("Residual")
plt.tight_layout()
plt.show()

plt.figure(figsize=(6,4))
plt.hist(res, bins=40)
plt.title("Histogram Residuals (Close) — TEST")
plt.tight_layout()
plt.show()

In [ ]:
# ===== Cell 21: barplot variable importance encoder & decoder =====
import matplotlib.pyplot as plt

def barplot_top(pairs, title):
    if not pairs:
        print(f"(i) No data for {title}")
        return
    names = [p[0] for p in pairs][::-1]
    vals  = [p[1] for p in pairs][::-1]
    plt.figure(figsize=(8, max(3, 0.4*len(names))))
    plt.barh(range(len(names)), vals)
    plt.yticks(range(len(names)), names)
    plt.title(title)
    plt.tight_layout()
    plt.show()

barplot_top(top_enc, "Encoder Variable Importance (avg |contribution|)")
barplot_top(top_dec, "Decoder Variable Importance (avg |contribution|)")

In [ ]:
# ===== Cell 22 (REPLACE): Temporal attention vs Date — robust untuk attention agregat =====
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def _safe_to_np(t):
    if t is None:
        return None
    a = t.detach().cpu().numpy()
    if a.ndim == 3 and a.shape[-1] == 1:
        a = a[..., 0]
    return a

def _get_attention_from_raw(raw):
    """Return attention with shape (N, dec_len, enc_len) if possible."""
    attn = None
    if isinstance(raw, dict):
        attn = raw.get("attention", None)
    if attn is None and isinstance(raw, list) and len(raw) > 0 and isinstance(raw[0], dict):
        attn = raw[0].get("attention", None)
    if attn is None:
        return None

    A = np.array(attn)
    if A.ndim == 4:        # (N, heads, dec_len, enc_len)
        A = A.mean(axis=1) # -> (N, dec_len, enc_len)
    elif A.ndim == 3:      # (N, dec_len, enc_len)
        pass
    elif A.ndim == 2:      # (N, enc_len) atau (dec_len, enc_len) tanpa batch
        # asumsikan sudah agregat → tambahkan dim batch & dec_len kalau perlu
        if A.shape[0] != A.shape[1]:      # (dec_len, enc_len)
            A = A[None, ...]              # -> (1, dec_len, enc_len)
        else:                             
            A = A[:, None, :]             # -> (N, 1, enc_len)
    else:
        return None
    return A

def plot_temporal_attention_by_date(
    dataloader,
    split_name="TEST",
    target_date=None,   # e.g. "2025-07-31"
    decoder_h=0,
    overlay_price=True
):
    # ambil raw + x + index
    raw, x, idx = tft.predict(dataloader, mode="raw", return_x=True, return_index=True)
    raw0 = raw[0] if isinstance(raw, list) and len(raw) > 0 else raw

    # ambil attention
    A = _get_attention_from_raw(raw0)
    aggregated = False
    if A is None:
        # fallback: interpret_output (agregat, tanpa batch per-sampel)
        try:
            interpv = tft.interpret_output(raw0, reduction="mean")
            A = interpv.get("attention", None)
            if A is None:
                print("(i) Attention tidak tersedia pada output ini.")
                return
            A = np.array(A)
            if A.ndim == 1: A = A[None, :]
            if A.ndim == 2: A = A[None, ...]  # -> (1, dec_len, enc_len)
            aggregated = True
            print("(i) Menggunakan attention *rata-rata* (agregat).")
        except Exception as e:
            print("(i) Attention tidak tersedia dan interpret_output gagal:", e)
            return

    # decoder t0 (waktu prediksi)
    dec_ti = _safe_to_np(x.get("decoder_time_idx"))
    if dec_ti is not None:
        dec_ti = dec_ti.astype(int)
        if dec_ti.ndim == 1:
            dec_ti = dec_ti[:, None]
        dec_t0 = dec_ti[:, 0]
    else:
        if isinstance(idx, (pd.DataFrame, pd.Series)) and "time_idx" in idx:
            dec_t0 = idx["time_idx"].to_numpy().astype(int).reshape(-1)
            dec_ti = dec_t0[:, None]
        else:
            raise RuntimeError("Tidak bisa memperoleh decoder time_idx dari x maupun idx.")

    N_x = len(dec_t0)                 # jumlah sampel di loader
    N_A = A.shape[0]                  # jumlah batch di attention

    # mapping time_idx -> Date / Close
    t2date  = dict(zip(df_tft["time_idx"].to_numpy(), pd.to_datetime(df_tft["Date"])))
    t2close = dict(zip(df_tft["time_idx"].to_numpy(), df_tft["Close"].to_numpy()))

    # pilih sampel index
    if target_date is not None:
        target_ts = pd.to_datetime(target_date)
        dec_dates = np.array([t2date.get(int(t), pd.NaT) for t in dec_t0], dtype="datetime64[ns]")
        valid = ~pd.isna(dec_dates)
        if valid.sum() == 0:
            print("Tidak ada sampel dengan decoder date yang valid.")
            return
        deltas = np.abs(dec_dates[valid] - target_ts)
        n_idx_x = np.arange(N_x)[valid][np.argmin(deltas)]
    else:
        target_ts = None
        n_idx_x = N_x - 1  # pakai yang terbaru

    # pilih horizon
    dec_len = A.shape[1]
    h = min(decoder_h, dec_len - 1)

    # siapkan encoder index vector & attention vector dengan panjang sama
    enc_len_att = A.shape[2]

    if N_A == N_x:
        # attention per-sampel tersedia → pilih baris n_idx_x
        att_vec = A[n_idx_x, h, :]  # (enc_len,)
        # ambil encoder_time_idx jika ada, kalau tidak rekonstruksi dari encoder_lengths
        enc_ti = _safe_to_np(x.get("encoder_time_idx"))
        if enc_ti is not None:
            if enc_ti.ndim == 1: enc_ti = enc_ti[None, :]
            enc_idx_vec = enc_ti[n_idx_x]
            # align kanan: ambil tail sepanjang enc_len_att
            enc_idx_vec = enc_idx_vec[-enc_len_att:]
        else:
            enc_len = _safe_to_np(x.get("encoder_lengths"))
            if enc_len is None:
                enc_len_i = training.max_encoder_length
            else:
                enc_len_i = int(enc_len.reshape(-1)[n_idx_x])
            # buat jendela encoder yang berakhir di dec_t0[n_idx_x]-1
            enc_idx_vec = np.arange(dec_t0[n_idx_x] - enc_len_att, dec_t0[n_idx_x], dtype=int)
    else:
        # attention agregat (batch=1) → pakai att_vec tunggal
        aggregated = True
        att_vec = A[0, h, :]  # (enc_len,)
        # rekonstruksi encoder index vektor dengan panjang = enc_len_att
        enc_idx_vec = np.arange(dec_t0[n_idx_x] - enc_len_att, dec_t0[n_idx_x], dtype=int)

    # mapping ke tanggal
    enc_dates = pd.to_datetime([t2date.get(int(t), pd.NaT) for t in enc_idx_vec])
    close_series = np.array([t2close.get(int(t), np.nan) for t in enc_idx_vec])

    plot_df = pd.DataFrame({"Date": enc_dates, "attention": att_vec, "Close": close_series})
    plot_df = plot_df.dropna(subset=["Date"]).sort_values("Date")
    if len(plot_df) == 0:
        print("(i) Tidak ada data attention valid untuk sampel ini.")
        return
    plot_df["attention_norm"] = plot_df["attention"] / (plot_df["attention"].max() + 1e-12)

    # plot
    fig, ax = plt.subplots(figsize=(12, 4))
    title_extra = " (agregat)" if aggregated else f" (sample idx={n_idx_x})"
    ax.plot(plot_df["Date"], plot_df["attention_norm"], label=f"Attention (norm), H{h+1}{title_extra}")
    ax.set_title(f"Temporal Attention vs Date — {split_name}\n{('target_date='+str(target_ts)) if target_ts is not None else 'latest'}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Attention (normalized)")
    ax.grid(True, alpha=0.3)

    if overlay_price:
        ax2 = ax.twinx()
        ax2.plot(plot_df["Date"], plot_df["Close"], alpha=0.35)
        ax2.set_ylabel("Close (overlay)")

    ax.legend(loc="upper left")
    plt.tight_layout()
    plt.show()

    # Top-10 tanggal paling “diperhatikan”
    topk = plot_df.sort_values("attention", ascending=False).head(10)
    print("Top-10 encoder dates by attention:")
    display(topk[["Date","attention"]])

In [ ]:
# ===== Cell 23: contoh penggunaan =====

# 1) Sampel TEST paling baru (default)
plot_temporal_attention_by_date(eval_test_loader, split_name="TEST", target_date=None, decoder_h=0, overlay_price=True)

# 2) (opsional) pilih tanggal spesifik untuk titik prediksi H1,
#    misal kamu pengin lihat prediksi untuk 2025-07-31
# plot_temporal_attention_by_date(eval_test_loader, split_name="TEST", target_date="2025-07-31", decoder_h=0, overlay_price=True)

# 3) (opsional) versi VAL
# plot_temporal_attention_by_date(eval_val_loader, split_name="VAL", target_date=None, decoder_h=0, overlay_price=True)

In [ ]:
torch.save(tft.state_dict(), "tft_model.pth")

